# **Sesión 3: Procesamiento Avanzado de Imágenes** ⚙️🖼️

## **Librerías**

In [1]:
import numpy as np
import cv2
import os
import imageio
from typing import List
import glob
import copy

## **Apartado A: Detección de esquinas**

El objetivo de este apartado es detectar las esquinas presentes en las imágenes de la carpeta ``data/source``.

1. **Tarea A.1**. Cree una nueva capeta llamada ``partA``, dentro de la carpeta  ``data``, con el objetivo de presentar en ella los resultados de esta parte de la práctica.
2. **Tarea A.2**. Defina y ejecute los dos métodos propuestos para cargar imágenes ``imageio_load_images()`` y ``opencv_load_images()``. Observe lo que ocurre al guardar ambas imágenes usando la misma función ``cv2.imwrite()``.
3. **Tarea A.3.** Defina la función ``harris_corner_detector()``, que servirá para la posterior aplicación sobre las imágenes de trabajo. 
4. **Tarea A.4.** Aplique la función ``harris_corner_detector()`` sobre las imágenes de trabajo. Asegúrese de que las imágenes quedan guardadas como se especifica en los comentarios.
5. **Tarea A.5.** Defina la función ``shi_tomasi_corner_detection()``, que servirá para la posterior aplicación sobre las imágenes de trabajo.
6. **Tarea A.6.** Aplique la función ``shi_tomasi_corner_detection()`` sobre las imágenes de trabajo. Asegúrese de que las imágenes quedan guardadas como se especifica en los comentarios.

### **Tarea A.1**. Cree una nueva capeta llamada ``partA``, dentro de la carpeta  ``data``, con el objetivo de presentar en ella los resultados de esta parte de la práctica.

In [2]:
# TODO Create a folder to save all partA results (inside data)
import os

folder_name = "partA"
folder_path = os.path.join("../data", folder_name)
os.makedirs(folder_path, exist_ok=True)


### **Tarea A.2**. Defina y ejecute los dos métodos propuestos para cargar imágenes ``imageio_load_images()`` y ``opencv_load_images()``. Observe lo que ocurre al guardar ambas imágenes usando la misma función ``cv2.imwrite()``.

In [3]:
# This initial part is to highlight that cv2.imshow() and cv2.imwrite() works well with previous BGR conversion
def imageio_load_images(filenames: List[str]) -> List:
    '''
    Load images using imageio.imread function (RGB)
    '''
    return [imageio.v2.imread(filename) for filename in filenames]

def opencv_load_images(filenames: List) -> List:
    '''
    Load images cv2.imread function (BGR)
    '''
    return [cv2.imread(filename) for filename in filenames]

# TODO Create two sorted lists with the paths of all images in the data/source folder using glob
source_folder = os.path.join("../data", "source")
source_paths = sorted(glob.glob(os.path.join(source_folder, "*")))  # Cambia "*.jpg" por la extensión correcta si es diferente
imageio_images = imageio_load_images(source_paths)
opencv_images = opencv_load_images(source_paths)

print(len(imageio_images))

destination_folder = "../data/partA"

cv2.imwrite(os.path.join(destination_folder, "imageio_blue_court.jpg"), imageio_images[3])
cv2.imwrite(os.path.join(destination_folder, "opencv_blue_court.jpg"), opencv_images[-1])

4


True

### **Tarea A.3.** Defina la función ``harris_corner_detector()``, que servirá para la posterior aplicación sobre las imágenes de trabajo. 

In [4]:
# TODO Define Harris Corner detection function
def harris_corner_detector(image: np.array, blockSize: int, ksize: int, k: float):
    '''
    image - Input image 
    blockSize - Size of neighborhood considered for corner detection
    ksize - Aperture parameter of the Sobel derivative used
    k - Harris detector free parameter in the equation.
    '''
    # TODO Input image to Harris corner detector should be grayscale 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # TODO Input image to Harris corner detector should be float32 type
    gray = np.float32(gray)
    # TODO Apply Harris corner detection
    harris = cv2.cornerHarris(src=gray, blockSize=blockSize, ksize=ksize, k=k)
    # Result is dilated for marking the corners, not important
    harris = cv2.dilate(harris, None)
    # TODO Threshold for an optimal value of 1% of maximal R value
    # If pixel value > 1% max value, yo should to hightlight this as a red corner
    threshold = 0.01 * harris.max()
    image[harris > threshold] = [0, 0, 255]
    return image

### **Tarea A.4.** Aplique la función ``harris_corner_detector()`` sobre las imágenes de trabajo. Asegúrese de que las imágenes quedan guardadas como se especifica en los comentarios.

In [5]:
# This section is designed to to change corner detection parameters for each image
# We want to save processed image in path: Lab3/data/partA/Harris_{save_name}.jpg

# First image
save_name = "geometry"
# TODO Copy first original image
image = cv2.imread("../data/source/0.jpg")
# TODO Apply Harris Corner Detection
harris_image = harris_corner_detector(image, blockSize=2, ksize=3, k=0.04)
# TODO Save final image in partA folder
cv2.imwrite(os.path.join("../data/oartA", f"Harris_{save_name}.jpg"), harris_image)

# Second image
save_name = "football"
# TODO Copy second original image
image = cv2.imread("../data/source/1.png")
# TODO Apply Harris Corner Detection
harris_image = harris_corner_detector(image, blockSize=2, ksize=3, k=0.04)
cv2.imwrite(os.path.join(destination_folder, f"Harris_{save_name}.jpg"), harris_image)

True

### **Tarea A.5.** Defina la función ``shi_tomasi_corner_detection()``, que servirá para la posterior aplicación sobre las imágenes de trabajo.

In [6]:
# TODO Define Shi-Tomasi corner detection function
def shi_tomasi_corner_detection(image: np.array, maxCorners: int, qualityLevel:float, minDistance: int, corner_color: tuple, radius: int):
    '''
    image - Input image
    maxCorners - Maximum number of corners to return. 
                 If there are more corners than are found, the strongest of them is returned. 
                 maxCorners <= 0 implies that no limit on the maximum is set and all detected corners are returned
    qualityLevel - Parameter characterizing the minimal accepted quality of image corners. 
                   The parameter value is multiplied by the best corner quality measure, which is the minimal eigenvalue or the Harris function response. 
                   The corners with the quality measure less than the product are rejected. 
                   For example, if the best corner has the quality measure = 1500, and the qualityLevel=0.01 , then all the corners with the quality measure less than 15 are rejected
    minDistance - Minimum possible Euclidean distance between the returned corners
    corner_color - Desired color to highlight corners in the original image
    radius - Desired radius (pixels) of the circle
    '''
    # TODO Input image to Tomasi corner detector should be grayscale 
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # TODO Apply cv2.goodFeaturesToTrack function
    corners = cv2.goodFeaturesToTrack(gray, maxCorners=maxCorners, qualityLevel=qualityLevel, minDistance=minDistance)
    # TODO Corner coordinates conversion to integers
    corners = np.intp(corners) if corners is not None else []
    for corner in corners:
        # Multidimensional array into flattened array, if necessary
        x, y = corner.ravel()
        # TODO Circle corners
        cv2.circle(image, (x, y), radius, corner_color, 2)

    return image

### **Tarea A.6.** Aplique la función ``shi_tomasi_corner_detection()`` sobre las imágenes de trabajo. Asegúrese de que las imágenes quedan guardadas como se especifica en los comentarios.

In [7]:
# This section is designed to to change corner detection parameters for each image
# We want to save processed image in path: Lab3/data/partA/Shi-Tomasi_{save_name}.jpg

# First image - Purple corners and radius = 4
save_name = "geometry"
# TODO Purple color in adequate color space
purple_color = (255, 0, 255)
# TODO Copy first original image
image = cv2.imread("../data/source/0.jpg")
# TODO Apply Shi-Tomasi corner detection
tomasi_image = shi_tomasi_corner_detection(image, maxCorners=50, qualityLevel=0.1, minDistance=10, corner_color=purple_color, radius=4)

# TODO Save final image in partA folder
cv2.imwrite(os.path.join(destination_folder, f"ShiTomasi_{save_name}.jpg"), tomasi_image)

# Second image - Orange corners and radius = 4
save_name = "football"
# TODO Orange color in adequate color space
orange_color = (0, 165, 255) 
# TODO Copy second original image
image = cv2.imread("../data/source/1.png")
tomasi_image = shi_tomasi_corner_detection(image, maxCorners=42, qualityLevel=0.1, minDistance=10, corner_color=orange_color, radius=4)
# TODO Save final image in partA folder
cv2.imwrite(os.path.join(destination_folder, f"ShiTomasi_{save_name}.jpg"), tomasi_image)

True

### **Pregunta A.1:** Realice la detección de esquinas en las otras dos imágenes de la carpeta ``data/source`` (cuyos nombres de guardado han de ser "sudoku" y "tennis") con el método de Harris

In [8]:
# Code by yourself
# Code by yourself

# Crear la carpeta de salida si no existe
output_dir = "../data/partA"
os.makedirs(output_dir, exist_ok=True)

# Tercera imagen - Sudoku
save_name = "sudoku"
# Copiar la tercera imagen original (indice 2 para "sudoku")
image = copy.copy(opencv_images[2])
# Aplicar la detección de esquinas de Harris
harris_image = harris_corner_detector(image, blockSize=2, ksize=3, k=0.04)
# Guardar la imagen procesada
cv2.imwrite(f"{output_dir}/{save_name}.jpg", harris_image)

# Cuarta imagen - Tennis
save_name = "tennis"
# Copiar la cuarta imagen original (indice 3 para "tennis")
image = copy.copy(opencv_images[3])
# Aplicar la detección de esquinas de Harris
harris_image = harris_corner_detector(image, blockSize=2, ksize=3, k=0.04)
# Guardar la imagen procesada
cv2.imwrite(f"{output_dir}/{save_name}.jpg", harris_image)

True

### **Pregunta A.2:** Realice la detección de esquinas en las otras dos imágenes de la carpeta ``data/source`` (cuyos nombres de guardado han de ser "sudoku" y "tennis") con el método de Shi-Tomasi

In [10]:
# Code by yourself
# Code by yourself
# Tercera imagen - Sudoku, con esquinas en azul y radio = 4
save_name = "sudoku"
blue_color = (255, 0, 0)  # Color azul en el espacio de color BGR
# Copiar la tercera imagen original
image = copy.copy(opencv_images[2])
# Aplicar la detección de esquinas de Shi-Tomasi
tomasi_image = shi_tomasi_corner_detection(
    image, maxCorners=50, qualityLevel=0.02, minDistance=55, 
    corner_color=blue_color, radius=4
)
# Guardar la imagen procesada
cv2.imwrite(f"{output_dir}/tomasi_image_{save_name}.jpg", tomasi_image)

# Cuarta imagen - Tennis, con esquinas en amarillo y radio = 4
save_name = "tennis"
yellow_color = (0, 255, 255)  # Color amarillo en el espacio de color BGR
# Copiar la cuarta imagen original
image = copy.copy(opencv_images[3])
# Aplicar la detección de esquinas de Shi-Tomasi
tomasi_image = shi_tomasi_corner_detection(
    image, maxCorners=45, qualityLevel=0.02, minDistance=10, 
    corner_color=yellow_color, radius=4
)
# Guardar la imagen procesada
cv2.imwrite(f"{output_dir}/tomasi_image_{save_name}.jpg", tomasi_image)

True